# Data Wrangling and Visualization workbook
This workbook will be used to generate the data file that will be ready for use in the Event Based Model. Along the way, you will need to implement some of the data cleaning, data wrangling and data visualization skills that you learned in the demonstration notebook using the WHO cases data.  In the project notebook below, you will find instructions for what to do in each step followed by an empty code cell to enter your work. Don't forget that you will need to import the appropriate packages for this work.



In [2]:
# Put your import steps here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Step 1 - Load in ADNIMERGE
The first step we must do is to identify the measurements that will be used as features in the Event Based Model. From [Alex Young's original paper](https://academic.oup.com/brain/article/137/9/2564/2848155), the features that are to be included are: 
* Cerebrospinal Fluid (CSF) INNO-BIA AlzBio3 immunoassay ('INNO')
  * Amyloid Beta 1-42
  * phosphorylated tau
  * total tau
* Volumetric measurements from **1.5T** magnetic resonance imaging (MRI)
  * Whole brain volume
  * Ventricular volume
  * Entorhinal cortex volume
  * Hippocampal volume
  * Middle temporal cortex volume
  * Fusiform cortex volume 
  * Annualised whole brain atrophy between 0 and **12 months** using Boundary Shift Integral (BSI)
  * Annualised hippocampal atrophy between 0 and **12 months** using Boundary Shift Integral (BSI)
* Cognitive measures
  * Mini mental state examination (MMSE)
  * ADAS-COG13
  * Rey Auditory Verbal Learning Test (RAVLT)

Many of these features should be available in the ADNI MERGE dataset. The file is donwloaded from Teams and is called `adnimerge_ideas_merge_26may2022.csv`. We have included both the data dictionary and the methods so that you can better understand how this spreadsheet was created and what it represents. Please load in the ADNIMERGE spreadsheet. 
Here are some of the questions:
* Which column identifies the subject?
* Can you identify the features above in the column? 
* Are any of the features above missing?


In [3]:
# 117  TAU_INNO                  float64
#  118  ABETA_INNO                float64
#  119  PTAU_INNO                 float64
 
#   55   WholeBrain                float64
#  53   Ventricles                float64
#   56   Entorhinal                float64
#  54   Hippocampus               float64
#   58   MidTemp                   float64
#  57   Fusiform                  float64
#  5   ANN_BBSI    8421 non-null   float64       
#  10  ANN_HBSI    4661 non-null   float64       

# 26   MMSE                      float64
#  24   ADAS13                    float64

#  27   RAVLT_immediate           float64
#  28   RAVLT_learning            float64
#  29   RAVLT_forgetting          float64
#  30   RAVLT_perc_forgetting     float64

In [4]:
# Your answer to Step 1
# Below put your code that will load up the ADNI MERGE spreadsheet
input_file = "data/adnimerge_ideas_merge_26may2022.csv" 
df_raw = pd.read_csv(input_file)

/Users/catriona/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20,105,116) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df_raw.info(max)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15856 entries, 0 to 15855
Data columns (total 120 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   RID                       int64  
 1   COLPROT                   object 
 2   ORIGPROT                  object 
 3   PTID                      object 
 4   SITE                      int64  
 5   VISCODE                   object 
 6   EXAMDATE                  object 
 7   DX_bl                     object 
 8   AGE                       float64
 9   PTGENDER                  object 
 10  PTEDUCAT                  int64  
 11  PTETHCAT                  object 
 12  PTRACCAT                  object 
 13  PTMARRY                   object 
 14  APOE4                     float64
 15  FDG                       float64
 16  PIB                       float64
 17  AV45                      float64
 18  FBB                       float64
 19  ABETA_ELECSYS             object 
 20  TAU_ELECSYS               o

## Step 2 - Filter data
The event based model primarily works on cross-sectional data. In the data, the features you have identified from ADNIMERGE should be coming from the **baseline** visit and from scans acquired on a **1.5T** scanner.  Figure out how to filter the rows according to these two criteria. The resulting data frame should have 818 rows remaining.

In [6]:
# Your answer to Step 2
# Below put your code that will filter the ADNI MERGE spreadsheet

# for column in df_raw:  
#     print(column)
#     print(df_raw[column].value_counts())
#     print('----')

fld = df_raw['FLDSTRENG'] #want 1.5T values only
vsc = df_raw['VISCODE'] #want bl values only

fld.unique() #array(['1.5 Tesla MRI', nan, '3 Tesla MRI'], dtype=object)
vsc.isna().value_counts() #no null values in vsc, so can separate out these rows already
fld.isna().value_counts() #9935 null values in fld so need to work on this

#remove NaN rows from fld
df_complete = df_raw.dropna(axis=0, subset=['FLDSTRENG'])
fld = df_complete['FLDSTRENG'] #want 1.5T values onlycr
fld.value_counts()

#group by vsc, select 'bl' then group by fld, select '1.5 Tesla MRI' only
df_by_vsc= df_complete.groupby(['VISCODE'])
df_bl=df_by_vsc.get_group('bl')
df_bl_fld= df_bl.groupby(['FLDSTRENG'])
df_bl_fld_opf=df_bl_fld.get_group('1.5 Tesla MRI')

#get info to check 818 rows
df_bl_fld_opf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818 entries, 0 to 7234
Columns: 120 entries, RID to PTAU_INNO
dtypes: float64(91), int64(5), object(24)
memory usage: 773.3+ KB


In [7]:
# df_bl_fld_opf.info(max)
# df_bl_fld_opf['RID'].unique()
# df_bl_fld_opf['EXAMDATE']

## Step 3 - Merging data
You should have identified that some features from the original list that are missing in the ADNI MERGE spreadsheet. There is another spreadsheet in the data folder called `ucl_bsi_ideas_merge_26may2022.csv` where you will find the additional data.

In order to combine data you need to *merge* the two data sets. This involves finding key identifiers (the "on" columns) that will correspond to the same subject in both spreadsheets, so that the columns can be combined together in one data frame. Please remember that we only need BSI values from 0 to 12 months, so think about what filtering you will need to do with the new spreadsheet that you are loading in before performing the merge.

In [8]:
# Your answer to Step 3
# Below put your code that will merge the ADNI MERGE spreadsheet
# with other ADNI data availale.

new_input_file = "data/ucl_bsi_ideas_merge_26may2022.csv" 
df_new_raw = pd.read_csv(new_input_file)

# df_new_raw.info()
# df_bl_fld_opf.info(max)

#create new dataframes as copies of original ones
df_new_date = df_new_raw.copy()
df_old_date = df_bl_fld_opf.copy()

df_new_date['EXAMDATE'] = pd.to_datetime(df_new_raw['EXAMDATE'])
df_old_date['EXAMDATE'] = pd.to_datetime(df_bl_fld_opf['EXAMDATE'])

#remove unnecessary columns
selected_columns = ['RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'ANN_BBSI', 'ANN_HBSI', 'MRFIELD', 'MRSEQUENCE', 'PROTOCOL', 'QC_PASS', 'REGRATING']
df_new_date_clean = df_new_date.loc[:, selected_columns]


In [9]:
df_new_raw.head(50)
df_new_date_clean.info(max)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13875 entries, 0 to 13874
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   RID         13875 non-null  int64         
 1   VISCODE     13875 non-null  object        
 2   VISCODE2    13875 non-null  object        
 3   EXAMDATE    13875 non-null  datetime64[ns]
 4   ANN_BBSI    8421 non-null   float64       
 5   ANN_HBSI    4661 non-null   float64       
 6   MRFIELD     13875 non-null  float64       
 7   MRSEQUENCE  13875 non-null  object        
 8   PROTOCOL    13875 non-null  object        
 9   QC_PASS     13875 non-null  int64         
 10  REGRATING   9320 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 1.2+ MB


In [10]:
#extract only the 1.5T scans
df_new_date_clean['MRFIELD'].value_counts()
df_new_clean_mri = df_new_date_clean[df_new_date_clean['MRFIELD'] == 1.5]
# df_new_clean_mri.info(max)

In [11]:
# df_new_raw.sort_values('RID') #need to merge using RID as this is the patient ID I believe
# df_new_raw['VISCODE2'].value_counts()

# #There are only a couple of those values so will delete them, including only sc, scmri, m06, m12, bl
df_new_vsc = df_new_clean_mri.groupby(['VISCODE2'])
# df_new_vsc_bl = df_new_vsc.get_group('bl')
# df_new_vsc_sc = df_new_vsc.get_group('sc')
# df_new_vsc_m06 = df_new_vsc.get_group('m06')
df_new_vsc_m12 = df_new_vsc.get_group('m12')

#combine relevant scans back into single dataframe and sort by RID
df_new_firstyear = pd.concat([df_new_vsc_m12]) #df_new_vsc_bl, df_new_vsc_sc, df_new_vsc_m06, 
df_new_firstyear = df_new_firstyear.sort_values('RID')

#outer merge of both dataframes by RID (which is the common date format column)
df_bothdata = pd.merge(df_new_firstyear, df_old_date,
                        how='outer',on=['RID'],
                        suffixes=['_old','_new'],indicator=True).sort_values('RID')

# df_bothdata.info(max)
df_bothdata['_merge'].value_counts()
df_bothdata = df_bothdata.reset_index(drop=True)

In [12]:
 df_bothdata.info(max)
# df_bothdata.head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 131 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   RID                       int64         
 1   VISCODE_old               object        
 2   VISCODE2                  object        
 3   EXAMDATE_old              datetime64[ns]
 4   ANN_BBSI                  float64       
 5   ANN_HBSI                  float64       
 6   MRFIELD                   float64       
 7   MRSEQUENCE                object        
 8   PROTOCOL                  object        
 9   QC_PASS                   float64       
 10  REGRATING                 float64       
 11  COLPROT                   object        
 12  ORIGPROT                  object        
 13  PTID                      object        
 14  SITE                      int64         
 15  VISCODE_new               object        
 16  EXAMDATE_new              datetime64[ns]
 17  DX_bl          

## Step 4 - Remove unneeded columns
After the merge, how many columns are there? There should be way more columns than what are needed for the event based model. So we can get rid of unwanted columns that we don't need for the replication. You should have:
* 14 feature columns from the list in Step 1
* Additional columns that should be include, such as:
    * subject identifiers (IDs, visit codes, exam dates, site, protocols)
    * demographics (Age, Gender)
    * diagnoses
    * _APOE_ genetic status
    * intracranial volume, to serve as a proxy for head size.
    * image quality control (QC) variables (such as `TEMPQC`,`REGRATING`)
    * anything else you find relevant. I chose to keep 49 columns, but you may have a few more or a few less. 

In [13]:
# #THESE ARE THE VALUES WE NEED BASED ON THE LIST IN STEP 1

# 117  TAU_INNO                  float64
#  118  ABETA_INNO                float64
#  119  PTAU_INNO                 float64
 
#   55   WholeBrain                float64
#  53   Ventricles                float64
#   56   Entorhinal                float64
#  54   Hippocampus               float64
#   58   MidTemp                   float64
#  57   Fusiform                  float64
#  5   ANN_BBSI    8421 non-null   float64       
#  10  ANN_HBSI    4661 non-null   float64       

# 26   MMSE                      float64
#  24   ADAS13                    float64

#  27   RAVLT_immediate           float64
#  28   RAVLT_learning            float64
#  29   RAVLT_forgetting          float64
#  30   RAVLT_perc_forgetting     float64

In [14]:
# Your answer to Step 4
# Below put your code that will remove 
# unnecessary columns from the data frame.

select = ['RID', 'AGE', 'EXAMDATE_old', 'EXAMDATE_new', 'VISCODE_old', 'VISCODE_new', 'VISCODE2', 'TEMPQC', 'QC_PASS', 'DX_bl', 'APOE4', 'MRSEQUENCE', 'PROTOCOL', 'COLPROT', 'ORIGPROT', 'PTID', 'SITE', 'REGRATING', 'ICV', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT', 'PTMARRY', 'TAU_INNO', 'ABETA_INNO', 'PTAU_INNO', 'WholeBrain', 'Ventricles', 'Entorhinal', 'Hippocampus', 'MidTemp', 'Fusiform', 'ANN_BBSI', 'ANN_HBSI', 'MMSE', 'ADAS13', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting']
df_both_clean = df_bothdata.loc[:, select]
df_both_clean.head(20)

,RID,AGE,EXAMDATE_old,EXAMDATE_new,VISCODE_old,VISCODE_new,VISCODE2,TEMPQC,QC_PASS,DX_bl,...,MidTemp,Fusiform,ANN_BBSI,ANN_HBSI,MMSE,ADAS13,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting
0,2,74.3,NaT,2005-08-09,NaN,bl,NaN,Pass,NaN,CN,...,27936.0,16559.0,NaN,NaN,28.0,18.67,44.0,4.0,6.0,54.54550
1,3,81.3,2006-09-12,2005-12-09,nv,bl,m12,Pass,1.0,AD,...,18422.0,15506.0,19.090141,0.229140,20.0,31.00,22.0,1.0,4.0,100.00000
2,4,67.5,2006-11-28,2005-08-11,nv,bl,m12,Pass,1.0,LMCI,...,19615.0,19036.0,6.027470,0.020969,27.0,21.33,37.0,7.0,4.0,36.36360
3,5,73.7,2006-09-05,2005-07-09,nv,bl,m12,Pass,1.0,CN,...,21614.0,24788.0,14.563351,0.056883,29.0,14.67,37.0,4.0,4.0,44.44440
4,6,80.4,2006-11-28,2005-11-29,nv,bl,m12,Pass,1.0,LMCI,...,17802.0,17963.0,-2.380889,NaN,25.0,25.67,30.0,1.0,5.0,83.33330
5,7,75.4,NaT,2005-06-10,NaN,bl,NaN,Pass,NaN,AD,...,15374.0,12063.0,NaN,NaN,20.0,40.33,17.0,2.0,3.0,75.00000
6,8,84.5,NaT,2005-09-19,NaN,bl,NaN,Pass,NaN,CN,...,20076.0,14043.0,NaN,NaN,28.0,7.00,51.0,7.0,3.0,25.00000
7,10,73.9,2006-11-09,2005-10-11,nv,bl,m12,Pass,1.0,AD,...,19741.0,16761.0,16.597478,0.269590,24.0,24.33,20.0,2.0,5.0,100.00000
8,14,78.5,2006-11-28,2005-04-11,nv,bl,m12,Pass,1.0,CN,...,17798.0,13779.0,1.803046,0.081444,29.0,8.33,45.0,6.0,4.0,36.36360
9,15,80.8,2006-10-17,2005-10-18,nv,bl,m12,Pass,1.0,CN,...,17273.0,14953.0,NaN,NaN,29.0,9.00,50.0,5.0,3.0,25.00000


## Step 5 - Identify complete case data
The Event Based Model requires all features to be present for an observation to be included. Please remove any rows where one of the features that you plan to put in the EBM has missing data.

Please remove any data which failed QC check. The key column to check is `TEMPQC`.

Answer a few questions:
* For each variable, how many subjects had missing data? 
* How many subjects remain? 
* How do the subjects break down across diagnosis?
* Within diagnosis, how do they breakdown in terms of sex, age, APOE status?
* How do these numbers compare to the paper? 

This should be your final data set, which should consist of 283 participants.

In [15]:
# Your answer to Step 5
# Below put your code to remove missing data
# and answer the descriptive statsitics queries
# around the final data set

#remove failed QC
df_both_clean.value_counts('TEMPQC')
df_both_qc = df_both_clean[df_both_clean['TEMPQC'] != 'Fail']
df_both_qc.value_counts('TEMPQC')

TEMPQC
Pass    665
dtype: int64

In [16]:
#How many subjects with missing data
df_both_qc.isna().sum()

RID                        0
AGE                        0
EXAMDATE_old             108
EXAMDATE_new               0
VISCODE_old              108
VISCODE_new                0
VISCODE2                 108
TEMPQC                     0
QC_PASS                  108
DX_bl                      0
APOE4                      0
MRSEQUENCE               108
PROTOCOL                 108
COLPROT                    0
ORIGPROT                   0
PTID                       0
SITE                       0
REGRATING                108
ICV                        0
PTGENDER                   0
PTEDUCAT                   0
PTETHCAT                   0
PTRACCAT                   0
PTMARRY                    0
TAU_INNO                 380
ABETA_INNO               380
PTAU_INNO                380
WholeBrain                 0
Ventricles                 4
Entorhinal                 0
Hippocampus                0
MidTemp                    0
Fusiform                   0
ANN_BBSI                 148
ANN_HBSI      

In [21]:
#drop rows with NaN values
df_both_complete = df_both_qc.dropna()
df_both_complete.shape
df_both_complete.head(20)
df_both_complete.info()


#DX breakdown amongst remaining patients
# df_both_complete['DX_bl'].value_counts()

#STudy had 129 LMCI, 92 CN, and 64 AD, I have 282 patients so I'm missing one somehow.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282 entries, 1 to 812
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   RID                    282 non-null    int64         
 1   AGE                    282 non-null    float64       
 2   EXAMDATE_old           282 non-null    datetime64[ns]
 3   EXAMDATE_new           282 non-null    datetime64[ns]
 4   VISCODE_old            282 non-null    object        
 5   VISCODE_new            282 non-null    object        
 6   VISCODE2               282 non-null    object        
 7   TEMPQC                 282 non-null    object        
 8   QC_PASS                282 non-null    float64       
 9   DX_bl                  282 non-null    object        
 10  APOE4                  282 non-null    float64       
 11  MRSEQUENCE             282 non-null    object        
 12  PROTOCOL               282 non-null    object        
 13  COLPR

In [22]:

import os
data_root = "/Users/catriona/src/TeamCoders/EBM/TeamCoders_Event_Based_Model/Data_Cleaning_And_Wrangling/data/"
df_both_complete.to_csv(os.path.join(data_root,"ebm_mine.csv"))

In [ ]:
df_both_complete_dx = df_both_complete.groupby('DX_bl')
df_LMCI = df_both_complete_dx.get_group('LMCI')
df_CN = df_both_complete_dx.get_group('CN')
df_AD = df_both_complete_dx.get_group('AD')


#Counts of age, gender, APOE within each DX group
df_age_lmci = df_LMCI.value_counts('AGE')
df_age_cn = df_CN.value_counts('AGE')
df_age_ad = df_AD.value_counts('AGE')

print(f"LMCI: {df_age_lmci}, \n CN: {df_age_cn}, \n AD: {df_age_ad} \n \n")

df_gender_lmci = df_LMCI.value_counts('PTGENDER')
df_gender_cn = df_CN.value_counts('PTGENDER')
df_gender_ad = df_AD.value_counts('PTGENDER')

print(f"LMCI: {df_gender_lmci}, \n CN: {df_gender_cn}, \n AD: {df_gender_ad} \n \n")

df_APOE_lmci = df_LMCI.value_counts('APOE4')
df_APOE_cn = df_CN.value_counts('APOE4')
df_APOE_ad = df_AD.value_counts('APOE4')

print(f"LMCI: {df_APOE_lmci}, \n CN: {df_APOE_cn}, \n AD: {df_APOE_ad} \n \n")



## Step 6 - Data review
We have looked at the final data set at a group level, but let's actually visualise the data. For the various features that are to be included in the EBM, create some plots of your choice to look at how these values differ between the cognitively normal individuals and those showing evidence of cognitive impairment. You can break this latter group into mild cognitive impairment and Alzheimer's disease if you wish. You can also look at how this varies with _APOE_ status.

In [ ]:
# Your answer to Step 6

# 117  TAU_INNO                  float64
#  118  ABETA_INNO                float64
#  119  PTAU_INNO                 float64
 
#   55   WholeBrain                float64
#  53   Ventricles                float64
#   56   Entorhinal                float64
#  54   Hippocampus               float64
#   58   MidTemp                   float64
#  57   Fusiform                  float64
#  5   ANN_BBSI    8421 non-null   float64       
#  10  ANN_HBSI    4661 non-null   float64       

# 26   MMSE                      float64
#  24   ADAS13                    float64

#  27   RAVLT_immediate           float64
#  28   RAVLT_learning            float64
#  29   RAVLT_forgetting          float64
#  30   RAVLT_perc_forgetting     float64


%matplotlib inline  

#Plot each tau in each group

fig=plt.figure(figsize=(12.0, 8.0))
# Plot one Cases versus population for males
ax1= fig.add_subplot(1,3,1)
# First plot - across all countries and yearsHere we plot bar using the plt.bar() function and it is set at the ax1 which is axes1 subplot variable
ax1.set_title('TAU_INNO by ventricular volume') 
ax1.plot(df_LMCI['Ventricles'],df_LMCI['TAU_INNO'],'.')
ax1.plot(df_AD['Ventricles'],df_AD['TAU_INNO'],'.')
ax1.plot(df_CN['Ventricles'],df_CN['TAU_INNO'],'x')

ax1= fig.add_subplot(1,3,2)
# Second plot - across all countries and yearsHere we plot bar using the plt.bar() function and it is set at the ax1 which is axes1 subplot variable
ax1.set_title('ABETA_INNO by ventricular volume') 
ax1.plot(df_LMCI['Ventricles'],df_LMCI['ABETA_INNO'],'.')
ax1.plot(df_AD['Ventricles'],df_AD['ABETA_INNO'],'.')
ax1.plot(df_CN['Ventricles'],df_CN['ABETA_INNO'],'x')

ax1= fig.add_subplot(1,3,3)
# Second plot - across all countries and yearsHere we plot bar using the plt.bar() function and it is set at the ax1 which is axes1 subplot variable
ax1.set_title('PTAU_INNO by ventricular volume') 
ax1.plot(df_LMCI['Ventricles'],df_LMCI['PTAU_INNO'],'.')
ax1.plot(df_AD['Ventricles'],df_AD['PTAU_INNO'],'.')
ax1.plot(df_CN['Ventricles'],df_CN['PTAU_INNO'],'x')

In [ ]:
#extract each APOE status
df_both_complete['APOE4'].value_counts()
df_apoe_0 = df_both_complete[df_both_complete['APOE4'] == 0.0]
df_apoe_1 = df_both_complete[df_both_complete['APOE4'] == 1.0]
df_apoe_2 = df_both_complete[df_both_complete['APOE4'] == 2.0]

#groupby is good for stats over a group, can convert numeric values into different data type (enumerate?)
#[13:24] Cash, Dave    df_adni = df_adni[df_adni['VISCODE']=='bl']
#can also use loc function
# df_adni = df_adni.loc[:,'VISCODE'=='bl']

# Scatter plot for df1
plt.scatter(df_apoe_0['ANN_HBSI'], df_apoe_0['ABETA_INNO'], label='APOE 0')

# Scatter plot for df2
plt.scatter(df_apoe_1['ANN_HBSI'], df_apoe_1['ABETA_INNO'], label='APOE 1')

# Scatter plot for df3
plt.scatter(df_apoe_2['ANN_HBSI'], df_apoe_2['ABETA_INNO'], label='APOE 2')

# Set labels and title
plt.xlabel('Month 12 Annualised Hippocampal BSI')
plt.ylabel('Amyloid Beta 1-42')
plt.title('Amyloid Beta 1-42 against Hippocampal BSI')

# Add legend
plt.legend()

# Display the plot
plt.show()